In [23]:
!pip install termcolor

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

In [24]:
import matplotlib.pyplot as plt #dataviz
import seaborn as sb # dataviz
from sklearn.model_selection import train_test_split # ferramenta de dados

from sklearn.linear_model import LinearRegression # regressão linear
from sklearn.linear_model import Ridge # regressão tipo ridge
from sklearn.linear_model import Lasso # regressão tipo lasso
from sklearn.linear_model import BayesianRidge # regressão bayesiana
from sklearn.linear_model import ElasticNet # regressão elastic-net

from sklearn.metrics import explained_variance_score as evs # métricas de acuracidadee
from sklearn.metrics import r2_score as r2 # qualidade da regressão R2


from termcolor import colored as cl # customização
sb.set_style('whitegrid') # método de plotagem
plt.rcParams['figure.figsize'] = (20, 10) # tamanho da plotagem

In [12]:
##importação de base de dados
house_train = pd.read_csv("/workspaces/codespaces-jupyter/data/train.csv")
house_test = pd.read_csv("/workspaces/codespaces-jupyter/data/test.csv")

house_train.set_index('Id', inplace = True)
house_test.set_index('Id', inplace = True)

#house_train.dropna(inplace = True)
#house_test.dropna(inplace = True)


In [13]:
house_train.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


Primeira leitura da base de dados

Veja que a variável objetivo aqui é a SalePrice, é o Y que precisamos prever.

In [14]:
house_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [15]:
house_train['MasVnrArea'].fillna(0, inplace = True)
house_train['MasVnrArea'] = pd.to_numeric(house_train['MasVnrArea'], errors = 'coerce')
house_train['MasVnrArea'] = house_train['MasVnrArea'].astype('int64')

In [16]:
house_train.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [17]:
##seleção das variáveis do modelo
X_var = house_train[['LotArea', 'MasVnrArea', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF']].values
y_var = house_train['SalePrice'].values

In [18]:
##split train-test
X_train, X_test, y_train, y_test = train_test_split(X_var, y_var, test_size = 0.2, random_state = 0)


In [20]:
# Modelagem

# Regressão Linear

ols = LinearRegression()            ##chama o algoritmo
fit = ols.fit(X_train, y_train)           ##ajusta o modelo
y_pred = ols.predict(X_test)      ##previsão do modelo

In [27]:
print(cl('Score de variância do modelo (EVS) é de: {}'.format(evs(y_test, y_pred)), attrs = ['bold']))

Score de variância do modelo (EVS) é de: 0.46613085299101487


In [28]:
print(cl('Indicador R2 do modelo é de: {}'.format(r2(y_test, y_pred)), attrs = ['bold']))


Indicador R2 do modelo é de: 0.46597766987177247


In [29]:
from sklearn.metrics import mean_absolute_error

print('Test MAE  : %.3f'%mean_absolute_error(y_test, y_pred))
print('Train MAE : %.3f'%mean_absolute_error(y_train, fit.predict(X_train)))

Test MAE  : 30548.808
Train MAE : 26901.936


In [30]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error

print('Test MSE  : %.3f'%mean_squared_error(y_test, y_pred))
print('Train MSE : %.3f'%mean_squared_error(y_train, fit.predict(X_train)))

Test MSE  : 3687878089.293
Train MSE : 1587190407.484


In [31]:
from sklearn.metrics import mean_squared_log_error

print("Mean Squared Log Error : {:.3f}".format(mean_squared_log_error(y_test, y_pred)))

Mean Squared Log Error : 0.059


In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [34]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [35]:
from sklearn.model_selection import cross_val_score

lin_scores = cross_val_score(fit, X_train, y_train,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [32345.97187752 40272.65981322 55786.24630644 44015.28092239
 38186.89802811 42176.53791117 42514.11316172 43738.67688224
 30237.76774982 30566.00129993]
Mean: 39984.015395257244
Standard deviation: 7325.357492773642


In [36]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

In [37]:
grid_search.best_estimator_

RandomForestRegressor(max_features=8, n_estimators=30, random_state=42)

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

44897.02588706725 {'max_features': 2, 'n_estimators': 3}
39180.626293933965 {'max_features': 2, 'n_estimators': 10}
37409.88992437812 {'max_features': 2, 'n_estimators': 30}
44852.22454220482 {'max_features': 4, 'n_estimators': 3}
40399.13364813221 {'max_features': 4, 'n_estimators': 10}
37800.4497310812 {'max_features': 4, 'n_estimators': 30}
46090.9540015484 {'max_features': 6, 'n_estimators': 3}
39790.06184517701 {'max_features': 6, 'n_estimators': 10}
37926.052854662885 {'max_features': 6, 'n_estimators': 30}
44195.63949848665 {'max_features': 8, 'n_estimators': 3}
38944.67934853504 {'max_features': 8, 'n_estimators': 10}
37146.88437459864 {'max_features': 8, 'n_estimators': 30}
41439.101784856866 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
38391.16901632475 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
45594.11063564054 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
39400.21813005788 {'bootstrap': False, 'max_features': 3, 'n_estimators': 1

In [38]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,param_bootstrap,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.006892,0.000345,0.000756,0.000032,2,3,NaN,"{'max_features': 2, 'n_estimators': 3}",-1.242067e+09,-2.604272e+09,...,-2.015743e+09,5.458347e+08,16,-5.628929e+08,-4.788093e+08,-5.037019e+08,-5.486323e+08,-5.632194e+08,-5.314512e+08,3.415899e+07
1,0.021157,0.000751,0.001362,0.000014,2,10,NaN,"{'max_features': 2, 'n_estimators': 10}",-1.017901e+09,-2.253037e+09,...,-1.535121e+09,5.061875e+08,7,-2.883084e+08,-2.609555e+08,-3.240203e+08,-2.711742e+08,-3.476178e+08,-2.984152e+08,3.263117e+07
2,0.061132,0.000659,0.003215,0.000084,2,30,NaN,"{'max_features': 2, 'n_estimators': 30}",-9.906308e+08,-2.086546e+09,...,-1.399500e+09,4.848515e+08,2,-2.232727e+08,-1.871743e+08,-2.206687e+08,-2.069964e+08,-2.633271e+08,-2.202878e+08,2.503837e+07
3,0.008988,0.000322,0.000749,0.000022,4,3,NaN,"{'max_features': 4, 'n_estimators': 3}",-1.816481e+09,-2.485901e+09,...,-2.011722e+09,5.276832e+08,15,-5.506426e+08,-5.013304e+08,-5.452660e+08,-5.506731e+08,-5.555462e+08,-5.406917e+08,1.994763e+07
4,0.032889,0.006198,0.001607,0.000262,4,10,NaN,"{'max_features': 4, 'n_estimators': 10}",-1.331078e+09,-2.165357e+09,...,-1.632090e+09,4.845382e+08,11,-3.156833e+08,-2.693059e+08,-3.230610e+08,-2.846653e+08,-3.699094e+08,-3.125250e+08,3.480806e+07
5,0.086544,0.008901,0.003361,0.000258,4,30,NaN,"{'max_features': 4, 'n_estimators': 30}",-1.055446e+09,-1.965590e+09,...,-1.428874e+09,4.471325e+08,3,-2.238699e+08,-1.850645e+08,-2.244010e+08,-2.071698e+08,-2.603809e+08,-2.201772e+08,2.469780e+07
6,0.010838,0.000086,0.000733,0.000015,6,3,NaN,"{'max_features': 6, 'n_estimators': 3}",-1.639030e+09,-2.884691e+09,...,-2.124376e+09,6.239309e+08,18,-5.786600e+08,-4.463520e+08,-4.606346e+08,-5.680829e+08,-5.798555e+08,-5.267170e+08,6.009712e+07
7,0.035591,0.002282,0.001462,0.000083,6,10,NaN,"{'max_features': 6, 'n_estimators': 10}",-1.161689e+09,-2.338791e+09,...,-1.583249e+09,5.487265e+08,9,-2.815992e+08,-2.579589e+08,-2.773243e+08,-2.741445e+08,-3.899698e+08,-2.961993e+08,4.756275e+07
8,0.103378,0.001046,0.003436,0.000381,6,30,NaN,"{'max_features': 6, 'n_estimators': 30}",-1.060237e+09,-2.200124e+09,...,-1.438385e+09,5.353852e+08,4,-2.177227e+08,-1.801044e+08,-2.080608e+08,-1.958259e+08,-2.681641e+08,-2.139756e+08,2.987336e+07
9,0.013036,0.000233,0.000728,0.000009,8,3,NaN,"{'max_features': 8, 'n_estimators': 3}",-1.745051e+09,-2.158324e+09,...,-1.953255e+09,6.036600e+08,14,-6.034392e+08,-4.402228e+08,-4.915214e+08,-5.369766e+08,-5.448835e+08,-5.234087e+08,5.474820e+07


In [39]:
import joblib

##salvar nome do arquivo
filename = "regressao_linear.joblib"

#dumo de persistencia
joblib.dump(fit, filename)

##carregar modelo treinado
loaded_model = joblib.load(filename)

##usar modelo treinado
y_predicted = loaded_model.predict(X_test)
y_predicted

array([289812.70246921, 144382.02661645, 125261.9982037 , 242106.84443753,
       112518.68469507, 123799.67572513, 236928.09123016, 158146.11214064,
       491224.93443336, 122264.00635054, 175096.2053781 , 211553.50458778,
       221929.6481219 , 123348.27662493, 137766.66598447, 166392.49371239,
       204932.35301824,  91883.75220794, 135387.54316797, 153761.26165757,
       178387.25389616, 127829.09307056, 126756.55142797, 136889.11898402,
       211666.23037598, 183630.35786285, 169316.67058431,  52882.63396861,
       279355.13904281, 133930.88862207, 242321.50345943, 188454.48246037,
       127213.7464101 , 285757.32741102, 308514.60980256, 162500.64708355,
       233239.39510116, 124855.89562214, 225506.24338283, 318436.72592347,
       221315.74113075, 133619.51791376, 152248.87142882, 283769.37645083,
       341643.95583666, 206894.70148081, 150000.20329535, 126956.98999764,
       190999.48618664,  58392.69720711, 348128.51246352, 159773.03583816,
       206581.05417836,  